# Experiment 002: Ensemble Best Per-N from Multiple Solutions

Combine bucket-of-chump and saspav solutions by taking the best configuration for each N.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import json

getcontext().prec = 30
scale_factor = 1

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(Decimal('0.0') * scale_factor), float(tip_y * scale_factor)),
            (float(top_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
            (float(top_w / Decimal('4') * scale_factor), float(tier_1_y * scale_factor)),
            (float(mid_w / Decimal('2') * scale_factor), float(tier_2_y * scale_factor)),
            (float(mid_w / Decimal('4') * scale_factor), float(tier_2_y * scale_factor)),
            (float(base_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(base_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(mid_w / Decimal('4')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(mid_w / Decimal('2')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(top_w / Decimal('4')) * scale_factor), float(tier_1_y * scale_factor)),
            (float(-(top_w / Decimal('2')) * scale_factor), float(tier_1_y * scale_factor)),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(str(max(bounds[2] - bounds[0], bounds[3] - bounds[1]))) / scale_factor

def load_solution(csv_path):
    """Load a solution CSV and return a dict of {n: [(x, y, deg), ...]}"""
    df = pd.read_csv(csv_path)
    df['x'] = df['x'].astype(str).str.strip().str.lstrip('s')
    df['y'] = df['y'].astype(str).str.strip().str.lstrip('s')
    df['deg'] = df['deg'].astype(str).str.strip().str.lstrip('s')
    df[['group_id', 'item_id']] = df['id'].str.split('_', n=2, expand=True)
    
    solution = {}
    for group_id, group_data in df.groupby('group_id'):
        n = int(group_id)
        trees = [(row['x'], row['y'], row['deg']) for _, row in group_data.iterrows()]
        solution[n] = trees
    
    return solution

def score_config(trees_data):
    """Score a single N configuration given list of (x, y, deg) tuples"""
    tree_list = [ChristmasTree(x, y, deg) for x, y, deg in trees_data]
    side = get_tree_list_side_length(tree_list)
    n = len(trees_data)
    return float(side ** 2 / Decimal(str(n)))

print('Functions defined.')

Functions defined.


In [2]:
# Load both solutions
boc_path = '/home/code/exploration/preoptimized/submission.csv'
saspav_path = '/home/code/exploration/saspav/santa-2025.csv'

print('Loading bucket-of-chump solution...')
boc_solution = load_solution(boc_path)
print(f'Loaded {len(boc_solution)} N configurations')

print('Loading saspav solution...')
saspav_solution = load_solution(saspav_path)
print(f'Loaded {len(saspav_solution)} N configurations')

Loading bucket-of-chump solution...


Loaded 200 N configurations
Loading saspav solution...


Loaded 200 N configurations


In [3]:
# Compare and select best per-N
ensemble_solution = {}
boc_wins = 0
saspav_wins = 0
ties = 0

print('Comparing solutions per N...')
for n in range(1, 201):
    boc_score = score_config(boc_solution[n])
    saspav_score = score_config(saspav_solution[n])
    
    if boc_score < saspav_score:
        ensemble_solution[n] = boc_solution[n]
        boc_wins += 1
    elif saspav_score < boc_score:
        ensemble_solution[n] = saspav_solution[n]
        saspav_wins += 1
    else:
        ensemble_solution[n] = boc_solution[n]  # tie goes to boc
        ties += 1
    
    if n <= 20 or n % 50 == 0:
        winner = 'boc' if boc_score <= saspav_score else 'saspav'
        print(f'  N={n:3d}: boc={boc_score:.6f}, saspav={saspav_score:.6f}, winner={winner}')

print(f'\nSummary: boc wins {boc_wins}, saspav wins {saspav_wins}, ties {ties}')

Comparing solutions per N...
  N=  1: boc=0.661250, saspav=0.661250, winner=saspav
  N=  2: boc=0.450779, saspav=0.450779, winner=boc
  N=  3: boc=0.434745, saspav=0.434745, winner=saspav
  N=  4: boc=0.416545, saspav=0.416545, winner=boc
  N=  5: boc=0.416850, saspav=0.416850, winner=boc
  N=  6: boc=0.399610, saspav=0.399610, winner=saspav
  N=  7: boc=0.399897, saspav=0.399897, winner=boc
  N=  8: boc=0.385407, saspav=0.385407, winner=saspav
  N=  9: boc=0.387415, saspav=0.387415, winner=boc
  N= 10: boc=0.376630, saspav=0.376630, winner=saspav
  N= 11: boc=0.374924, saspav=0.375736, winner=boc
  N= 12: boc=0.372724, saspav=0.372724, winner=boc
  N= 13: boc=0.372294, saspav=0.372294, winner=saspav
  N= 14: boc=0.369543, saspav=0.370454, winner=boc
  N= 15: boc=0.379203, saspav=0.379203, winner=saspav
  N= 16: boc=0.374128, saspav=0.374128, winner=saspav
  N= 17: boc=0.370040, saspav=0.370040, winner=boc
  N= 18: boc=0.368771, saspav=0.368771, winner=boc
  N= 19: boc=0.368615, saspav

  N= 50: boc=0.360753, saspav=0.360753, winner=saspav


  N=100: boc=0.345531, saspav=0.345531, winner=boc


  N=150: boc=0.337064, saspav=0.337064, winner=boc


  N=200: boc=0.337549, saspav=0.337564, winner=boc

Summary: boc wins 109, saspav wins 88, ties 3


In [4]:
# Calculate total ensemble score
ensemble_total = sum(score_config(ensemble_solution[n]) for n in range(1, 201))
boc_total = sum(score_config(boc_solution[n]) for n in range(1, 201))
saspav_total = sum(score_config(saspav_solution[n]) for n in range(1, 201))

print(f'bucket-of-chump total: {boc_total:.6f}')
print(f'saspav total: {saspav_total:.6f}')
print(f'Ensemble total: {ensemble_total:.6f}')
print(f'Improvement over boc: {boc_total - ensemble_total:.6f}')
print(f'Target: 68.919154')
print(f'Gap to target: {ensemble_total - 68.919154:.6f}')

bucket-of-chump total: 70.647327
saspav total: 70.658891
Ensemble total: 70.647306
Improvement over boc: 0.000021
Target: 68.919154
Gap to target: 1.728152


In [5]:
# Create submission CSV
rows = []
for n in range(1, 201):
    for i, (x, y, deg) in enumerate(ensemble_solution[n]):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': f's{x}',
            'y': f's{y}',
            'deg': f's{deg}'
        })

submission_df = pd.DataFrame(rows)
submission_df.to_csv('/home/submission/submission.csv', index=False)
submission_df.to_csv('submission.csv', index=False)
print(f'Saved submission with {len(submission_df)} rows')
print(submission_df.head(10))

Saved submission with 20100 rows
      id                          x                         y  \
0  001_0  s-48.19608619421424577922  s58.77098461521422478882   
1  002_0        s0.1540970696213643     s-0.03854074269478543   
2  002_1       s-0.1540970696213643      s-0.5614592573052146   
3  003_0    s1.12365581614030096702   s0.78110181599256300888   
4  003_1    s1.23405569584216001644   s1.27599950066375900093   
5  003_2    s0.64171464022907498403   s1.18045856661338111060   
6  004_0       s-0.3247477895908756       s0.1321099780911856   
7  004_1        s0.3153543462411342       s0.1321099780664757   
8  004_2        s0.3247477895908756      s-0.7321099780664753   
9  004_3       s-0.3153543481363217      s-0.7321099780911857   

                         deg  
0   s45.00000000000000000000  
1        s203.62937773065684  
2        s23.629377730656792  
3  s111.12513229289299943048  
4   s66.37062226934300213088  
5  s155.13405193710082130565  
6         s156.3706221456364  
7  

In [6]:
# Save metrics
metrics = {
    'cv_score': ensemble_total,
    'boc_score': boc_total,
    'saspav_score': saspav_total,
    'improvement_over_boc': boc_total - ensemble_total,
    'boc_wins': boc_wins,
    'saspav_wins': saspav_wins,
    'ties': ties
}

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f'Saved metrics: {metrics}')

Saved metrics: {'cv_score': 70.64730613290918, 'boc_score': 70.6473268976368, 'saspav_score': 70.65889102126869, 'improvement_over_boc': 2.076472762269077e-05, 'boc_wins': 109, 'saspav_wins': 88, 'ties': 3}
